In [67]:
import requests
from imageio import imread
import matplotlib.pyplot as plt
import cv2
import numpy as np
import coordinate
import folium
%matplotlib inline

In [2]:
org = {'lat': 21.0369098, 'lon': 105.8019358}
dst = {'lat': 21.031302, 'lon': 105.81302939999999}

In [50]:
agents = [
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:54.0) Gecko/20100101 Firefox/69.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:61.0) Gecko/20100101 Firefox/69.0",
    "Mozilla/5.0 (X11; Linux i586; rv:31.0) Gecko/20100101 Firefox/69.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:67.0) Gecko/20100101 Firefox/67.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0",
    "Mozilla/5.0 (X11; U; Linux Core i7-4980HQ; de; rv:32.0; compatible; JobboerseBot; http://www.jobboerse.com/bot.htm) Gecko/20100101 Firefox/38.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1 Safari/605.1.15"
]

In [112]:

def get_image_tile(org, dst, z, x, y):
    url = f"https://www.google.com/maps/vt?pb=!1m4!1m3!1i{z}!2i{x}!3i{y}!2m2!2sm!3i510224828!2m12!1i2!2sspotlight!8m9!8m8!1m3!3m2!3d{org['lat']}!4d{org['lon']}!1m3!3m2!3d{dst['lat']}!4d{dst['lon']}!3m5!2sen!12m3!1i68!2m1!1sset!5m0"
    
    print(url)
    count_fail = 0
    while True:
        try:
            headers = {
                'User-Agent': np.random.choice(agents, 1)[0]
            }
            resp = requests.get(url, headers=headers, stream=True).raw
            image = np.asarray(bytearray(resp.read()), dtype="uint8")
            image = cv2.imdecode(image, cv2.IMREAD_COLOR)[:, :, [2, 1, 0]]  # rgb
            print("fail time: ", count_fail)
            return image
        except:
            count_fail += 1


def find_skeleton3(img_org):
    img = img_org.copy()
    skeleton = np.zeros(img.shape,np.uint8)
    eroded = np.zeros(img.shape,np.uint8)
    temp = np.zeros(img.shape,np.uint8)

    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))

    iters = 0
    while(True):
        cv2.erode(img, kernel, eroded)
        cv2.dilate(eroded, kernel, temp)
        cv2.subtract(img, temp, temp)
        cv2.bitwise_or(skeleton, temp, skeleton)
        img, eroded = eroded, img # Swap instead of copy

        iters += 1
        if cv2.countNonZero(img) == 0:
            return (skeleton,iters)
        
            
def batch_points2deg(points, xtile, ytile, tile_size=512):
    """Faster `point_in_map_to_lat_lon` with numpy broadcasting"""
    points = np.array(points)
    xs = points[:, 1]
    ys = points[:, 0]
    xtiles = (xs / tile_size) + xtile
    ytiles = (ys / tile_size) + ytile
    return xs, ys, coordinate.batch_num2deg(xtiles, ytiles)

In [58]:
res = get_image_tile(org, dst, 16, 52028, 28849)

https://www.google.com/maps/vt?pb=!1m4!1m3!1i16!2i52028!3i28849!2m2!2sm!3i510224828!2m12!1i2!2sspotlight!8m9!8m8!1m3!3m2!3d21.0168438!4d105.81562559999999!1m3!3m2!3d21.0307282!4d105.8523393!3m5!2sen!12m3!1i68!2m1!1sset!5m0
fail time:  0


## Find full geometry from A to B

In [122]:
## Addition color

blue = (102, 157, 246)
orange = (255, 171, 52)
orange2 = (255, 170, 51)
red = (221, 25, 29)

traffic_colors = [blue, orange, orange2, red]

def check_tile_new(tile_img):
    res = np.zeros(tile_img.shape[:2]).astype(np.uint8)
    for color in traffic_colors:
        fil = ((tile_img == color).sum(axis=-1) == 3).astype(np.uint8)  # blue
        fil = cv2.morphologyEx(fil, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
        res = cv2.bitwise_or(res, fil)
    if np.sum(res) > 50:
        return res
        
    return None

In [116]:
def check_tile(tile_img):
    fil = ((tile_img == (102, 157, 246)).sum(axis=-1) == 3).astype(np.uint8)  # blue
    fil = cv2.morphologyEx(fil, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
    if np.sum(fil) > 50:
        return fil
        
    return None

def find_skeleton3(img_org):
    img = img_org.copy()
    skeleton = np.zeros(img.shape,np.uint8)
    eroded = np.zeros(img.shape,np.uint8)
    temp = np.zeros(img.shape,np.uint8)

    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))

    iters = 0
    while(True):
        cv2.erode(img, kernel, eroded)
        cv2.dilate(eroded, kernel, temp)
        cv2.subtract(img, temp, temp)
        cv2.bitwise_or(skeleton, temp, skeleton)
        img, eroded = eroded, img # Swap instead of copy

        iters += 1
        if cv2.countNonZero(img) == 0:
            return (skeleton,iters)
        

def find_box_around(x, y, contain_boxes, org, dst, z=16):
    for x_tile in range(x-1, x+2):
        for y_tile in range(y-1, y+2):
            if (x_tile, y_tile) not in contain_boxes:
                img = get_image_tile(org, dst, z, x_tile, y_tile)
                cv2.imwrite(f"test_imgs/{x_tile}_{y_tile}.png", img[:, :, ::-1])
                check = check_tile_new(img)
                if check is not None:
                    # find skeleton of img_tile
                    ske = find_skeleton3(check)[0]
                    xs, ys = np.where(ske != 0)
                    points = []
                    for x in np.unique(xs):
                        yyy = ys[xs == x].mean().astype(int)
                        points.append((x, yyy))
                    xs, ys, (coord_x, coord_y) = batch_points2deg(points, x_tile, y_tile, tile_size=256)
                    coords = np.hstack((coord_x[:, None], coord_y[:, None]))
                    return x_tile, y_tile, coords

    return None  # No new box around


def find_route(org, dst, z=16):
    coordinate.n = 2 ** z
    start_x, start_y = coordinate.deg2num(org['lat'], org['lon'])
    
    contain_boxes = [(start_x, start_y)]
    coords = []
    while True:
        res = find_box_around(start_x, start_y, contain_boxes, org, dst, z)
        if res is not None:
            contain_boxes.append(res[:-1])
            print("????", res[:-1])
            coords += list(res[-1])
            start_x, start_y = res[:-1]
            continue
        else:
            break
    return contain_boxes, coords


In [105]:
# 'org': (21.0168438, 105.81562559999999),
# 'dst': (21.0307282, 105.8523393)

In [106]:
org['lat'] = 21.0168438
org['lon'] = 105.81562559999999
dst['lat'] = 21.0307282
dst['lon'] = 105.8523393
z = 16

In [107]:
coordinate.n = 2 ** z
start_x, start_y = coordinate.deg2num(org['lat'], org['lon'])

In [108]:
print(start_x, start_y)

52031 28853


In [124]:
routes = find_route(org, dst, z=16)

https://www.google.com/maps/vt?pb=!1m4!1m3!1i16!2i52030!3i28852!2m2!2sm!3i510224828!2m12!1i2!2sspotlight!8m9!8m8!1m3!3m2!3d21.0168438!4d105.81562559999999!1m3!3m2!3d21.0307282!4d105.8523393!3m5!2sen!12m3!1i68!2m1!1sset!5m0
fail time:  1
https://www.google.com/maps/vt?pb=!1m4!1m3!1i16!2i52030!3i28853!2m2!2sm!3i510224828!2m12!1i2!2sspotlight!8m9!8m8!1m3!3m2!3d21.0168438!4d105.81562559999999!1m3!3m2!3d21.0307282!4d105.8523393!3m5!2sen!12m3!1i68!2m1!1sset!5m0
fail time:  2
https://www.google.com/maps/vt?pb=!1m4!1m3!1i16!2i52030!3i28854!2m2!2sm!3i510224828!2m12!1i2!2sspotlight!8m9!8m8!1m3!3m2!3d21.0168438!4d105.81562559999999!1m3!3m2!3d21.0307282!4d105.8523393!3m5!2sen!12m3!1i68!2m1!1sset!5m0
fail time:  0
https://www.google.com/maps/vt?pb=!1m4!1m3!1i16!2i52031!3i28852!2m2!2sm!3i510224828!2m12!1i2!2sspotlight!8m9!8m8!1m3!3m2!3d21.0168438!4d105.81562559999999!1m3!3m2!3d21.0307282!4d105.8523393!3m5!2sen!12m3!1i68!2m1!1sset!5m0
fail time:  0
???? (52031, 28852)
https://www.google.com/maps/vt?p

fail time:  0
???? (52035, 28851)
https://www.google.com/maps/vt?pb=!1m4!1m3!1i16!2i52034!3i28851!2m2!2sm!3i510224828!2m12!1i2!2sspotlight!8m9!8m8!1m3!3m2!3d21.0168438!4d105.81562559999999!1m3!3m2!3d21.0307282!4d105.8523393!3m5!2sen!12m3!1i68!2m1!1sset!5m0
fail time:  1
https://www.google.com/maps/vt?pb=!1m4!1m3!1i16!2i52034!3i28852!2m2!2sm!3i510224828!2m12!1i2!2sspotlight!8m9!8m8!1m3!3m2!3d21.0168438!4d105.81562559999999!1m3!3m2!3d21.0307282!4d105.8523393!3m5!2sen!12m3!1i68!2m1!1sset!5m0
fail time:  3
https://www.google.com/maps/vt?pb=!1m4!1m3!1i16!2i52035!3i28852!2m2!2sm!3i510224828!2m12!1i2!2sspotlight!8m9!8m8!1m3!3m2!3d21.0168438!4d105.81562559999999!1m3!3m2!3d21.0307282!4d105.8523393!3m5!2sen!12m3!1i68!2m1!1sset!5m0
fail time:  0
https://www.google.com/maps/vt?pb=!1m4!1m3!1i16!2i52036!3i28850!2m2!2sm!3i510224828!2m12!1i2!2sspotlight!8m9!8m8!1m3!3m2!3d21.0168438!4d105.81562559999999!1m3!3m2!3d21.0307282!4d105.8523393!3m5!2sen!12m3!1i68!2m1!1sset!5m0
fail time:  2
https://www.google

In [126]:
mymap = folium.Map((org['lat'], org['lon']), min_zoom=12)
for marker in np.array(routes[1]):
    folium.Circle(marker, 2).add_to(mymap)
# folium.PolyLine(np.array(routes[1])).add_to(mymap)
mymap

In [111]:
mymap = folium.Map((org['lat'], org['lon']), min_zoom=12)
for marker in np.array(routes[1]):
    folium.Marker(marker).add_to(mymap)
# folium.PolyLine(np.array(routes[1])).add_to(mymap)
mymap

## Addition color